In [ ]:
# Notebook Purpose: - Clean 2023 environmental dataset. - Standardise units and flags. - Prepare for stacking.

# Assumptions: 
# - Raw data contains mixed boolean types. 
# - Numeric fields may contain excessive decimals. 
# - Compliance fields may contain nulls.

# Output: - Cleaned 2023 dataset ready for stacking


In [ ]:
import pandas as pd 
df_2023 = pd.read_csv("../data/env_2023_60.csv")
df_2023



In [ ]:
# Data Inspection Utility. Purpose: Provides a structured, repeatable data quality overview of the raw 2023 dataset 
# before any cleaning or transformation is applied. This function performs structural, completeness, duplication, and schema validation checks to support defensible preprocessing. 
# Note: This function does NOT modify the dataset. It is purely diagnostic.

def inspect_df(df_2023: pd.DataFrame, sample_size: int = 5):
    def section(title: str):
        print("\n" + "=" * 80)
        print(title)
        print("=" * 80)

    section("SHAPE & STRUCTURE")
    print("Rows, Columns:", df_2023.shape)
    print("Columns:", df_2023.columns.tolist())
    print("Index:", df_2023.index)

    section("DATA TYPES & NULL OVERVIEW")
    df_2023.info()

    section("ROW-LEVEL SANITY CHECKS")
    display(df_2023.head(sample_size))
    display(df_2023.tail(sample_size))
    if len(df_2023) > 0:
        display(df_2023.sample(min(sample_size, len(df_2023))))

    section("MISSING VALUES")
    missing_counts = df_2023.isna().sum()
    missing_pct = df_2023.isna().mean().sort_values(ascending=False)
    print(missing_counts[missing_counts > 0])
    print(missing_pct[missing_pct > 0])

    section("DUPLICATES")
    print("Total duplicated rows:", df_2023.duplicated().sum())

    section("NUMERIC COLUMNS")
    num_df_2023 = df_2023.select_dtypes(include="number")
    if not df_2023.empty:
        display(num_df_2023.describe().T)

    section("CATEGORICAL COLUMNS")
    cat_df_2023 = df_2023.select_dtypes(include=["object", "category"])
    if not cat_df_2023.empty:
        print(cat_df_2023.nunique().sort_values())
        display(cat_df_2023.iloc[:, 0].value_counts(dropna=False).head(10))

    section("SCHEMA SNAPSHOT")
    print(df_2023.dtypes)

    print("\nINSPECTION COMPLETE — NO DATA MODIFIED")
    
inspect_df(df_2023)

In [ ]:
# Looking at column names to determine whether they need reording/renaming
df_2023.columns.tolist()

In [ ]:
# Mapping the new column names 
rename_map = {
    "site": "Site",
    "year": "Year",
    "month": "Month",
    "energy_mwh": "Energy_kWh",
    "water_m3": "Water_m3",
    "waste_tonnes": "Waste_tonnes",
    "ghg_tonnes": "CO2_tonnes",
    "incidents": "Environmental_incidents",
}

In [ ]:
# Standardising column names 
df_2023 = df_2023.rename(columns=rename_map)

In [ ]:
# Arrange columns in a consistent, logical order
# to improve readability and ensure compatibility
# with downstream stacking and SQL ingestion.
target_order = [
    "Site",
    "Year",
    "Month",
    "Energy_kWh",
    "Water_m3",
    "Waste_tonnes",
    "CO2_tonnes",
    "Environmental_incidents",
]
# Apply the column order to the DataFrame
df_2023 = df_2023[target_order]

In [ ]:
# Quick inspection to confirm new column order
df_2023.columns.tolist()
df_2023.head()

In [ ]:
# Initialise recycled indicator as binary (0 = no, 1 = yes).
# Column is populated later and may be cast to boolean for analysis.
df_2023["Recycled_percent"] = 0

# Initialise compliance score placeholder.
# Final score is calculated in the compliance review stage.
df_2023["Compliance_score"] = 0


In [ ]:
# Verify that the changes have been applied correctly
df_2023[["Recycled_percent", "Compliance_score"]].head()

In [ ]:
# Define measurement columns used to assess data availability.
# For each metric, a corresponding *_recorded flag is created
# to indicate whether a valid (non-null, non-zero) value exists.
measurement_cols = [
    "Energy_kWh",
    "Water_m3",
    "Waste_tonnes",
    "CO2_tonnes",
    "Environmental_incidents",
    "Recycled_percent",
    "Compliance_score",
]

In [ ]:
# Create recorded flags: True if value is present and non-zero
for col in measurement_cols:
    df_2023[f"{col}_recorded"] = df_2023[col].notna() & (df_2023[col] != 0)


In [ ]:
# Cast recorded flags to boolean for consistency and clarity
for col in measurement_cols:
    df_2023[f"{col}_recorded"] = df_2023[f"{col}_recorded"].astype(bool)


In [ ]:
# Override logic for environmental incidents:
# Zero incidents is a valid recorded value, so completeness is
# based on presence (non-null) rather than non-zero.

df_2023["Environmental_incidents_recorded"] = (
    df_2023["Environmental_incidents"].notna()
)


In [ ]:
# Quick sanity check to confirm incident completeness logic
df_2023.loc[
    df_2023["Environmental_incidents"] == 0,
    ["Environmental_incidents", "Environmental_incidents_recorded"]
].head()


In [ ]:
# Standardise site identifiers for consistency and easier querying in SQL Server 
site_map = { 
    'SITE A' : 'A', 
    'SITE B' : 'B', 
    'SITE C' : 'C', 
    'SITE D' : 'D', 
    'SITE E' : 'E' 
} 

df_2023['Site'] = df_2023['Site'].replace(site_map)
df_2023.head()

In [ ]:
# Filtering on months where there were additional but incomplete records.
df_2023[df_2023['Month'] == 10]

In [ ]:
# Filtering on Site where there were additional but incomplete records.
df_2023[df_2023['Site'] == 'C']

In [ ]:
# Define aggregation logic for monthly consolidation:
# - Consumption and emissions are summed
# - Percentage and score metrics are averaged
# - *_recorded flags use 'any' to indicate data presence within the mon
agg_rules = {
    "Energy_kWh": "sum",
    "Water_m3": "sum",
    "Waste_tonnes": "sum",
    "CO2_tonnes": "sum",
    "Environmental_incidents": "sum",
    "Recycled_percent": "mean",
    "Compliance_score": "mean",

    "Energy_kWh_recorded": "any",
    "Water_m3_recorded": "any",
    "Waste_tonnes_recorded": "any",
    "Recycled_percent_recorded": "any",
    "CO2_tonnes_recorded": "any",
    "Compliance_score_recorded": "any",
    "Environmental_incidents_recorded": "any",
}

In [ ]:
# Aggregate to a single record per Site–Year–Month
df_clean = (
    df_2023
    .groupby(["Site", "Year", "Month"], as_index=False)
    .agg(agg_rules)
)

In [ ]:
# Validate that monthly consolidation was successful
# (no duplicate Site–Year–Month combinations)
df_clean.duplicated(["Site", "Year", "Month"]).sum()

In [ ]:
# Confirm expected completeness flag columns exist post-aggregation
"Environmental_incidents_recorded" in df_clean.columns

In [ ]:
# Quick inspection of final structure
df_clean.head()
df_clean.columns.tolist()

In [ ]:
# Creating a copy to save for later when I will stack the 2023-2025 cleaned datasets. 
df_2023_clean = df_clean.copy()

In [ ]:
# Persist cleaned 2023 data to disk to support a reproducible
# multi-year environmental data pipeline
df_2023_clean.to_csv(
    "Environmental_data_2023_clean.csv", 
    index=False
)

In [ ]:
# Some months contain multiple partial records for the same site.
# These are consolidated into a single monthly record to ensure
# one row per Site–Year–Month for reporting and SQL compatibility.
#
# Raw (pre-aggregated) data is retained separately for traceability.
# Aggregation rules are chosen to preserve measurement meaning.